# Novel Inertia Based Human Pose Estimation Using CNN

In [1]:
#setup
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import scipy.io as sio
from utils import *

#need to have these two lines to work on my ancient 1060 3gb
#  https://stackoverflow.com/questions/43990046/tensorflow-blas-gemm-launch-failed
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
%load_ext autoreload
%autoreload 2
%autosave 180

Autosaving every 180 seconds


In [ ]:
#Import and augment data by rotating about y axis

from utils import *

f1 = "simulation/data/traj_9DOF_rel2start100k.txt"
f2 = "simulation/data/jointPos_9DOF_rel2start100k.txt"
numTraj = 100000
t, jointPos = add_body_rotation(f1, f2, numTraj, mult = 1)

# f3 = "simulation/data/traj_9DOF_250k.txt"
# f4 = "simulation/data/jointPos_9DOF_250k.txt"
# numTraj = 250000
# t2, jointPos2 = add_body_rotation(f3, f4, numTraj, mult = 1)


# print(jointPos[0])

In [ ]:
#combine traj and jointpos from the two datasets

# print(np.shape(t),np.shape(t2))
# print(np.shape(jointPos),np.shape(jointPos2))

tCombined = np.concatenate((t,t2), axis = 0)
jointPosCombined = np.concatenate((jointPos,jointPos2), axis = 0)

print(np.shape(tCombined), np.shape(jointPosCombined))

t = tCombined
jointPos = jointPosCombined

In [ ]:
#find and fix errors in data
err = np.argwhere(np.abs(jointPos) == 0)[:,0]
over_extended = np.argwhere(np.abs(jointPos) > 360)

print(err)
print(over_extended)

# Remove cells with errors instead of doubling existing cells
jointPos = np.delete(jointPos,err,axis = 0)
t = np.delete(t,err, axis = 0)

#replace empty trials with data from existing trials -> not a good idea...?
# for i in err[:,0]:
# #     print(jointPos[i])
#     randIndex = int(np.floor(np.random.rand()*np.shape(jointPos)[0]))
#     jointPos[i] = jointPos[randIndex]
#     t[i] = t[randIndex]

print(np.shape(jointPos))
print(np.shape(t))

# np.save("simulation/data/traj_combined", t)
# np.save("simulation/data/jointPos_combined", jointPos)

In [ ]:
#load pre-rotated dataset

t = np.load("simulation/data/traj_combined.npy")
jointPos = np.load("simulation/data/jointPos_combined.npy")

In [ ]:
from network import Net4 #for 10DOF model

np.random.seed(1337)

#convert data from numpy to tensors
x_train = tf.convert_to_tensor(t,np.float32)
y_train = tf.convert_to_tensor(jointPos,np.float32)

# print(tf.shape(x_train))
# print(x_train[0,:,1])

In [ ]:
model = Net4()

runLen = 100    

def scheduler(epoch, lr):
#     part1 = 2*runLen//3
#     part2 = 5*runLen//6 #net1

    part1 = runLen//3
    part2 = 2*runLen//3 #net2

    if epoch < part1:
        lr = 0.01
        return lr
    if epoch >= part1 and epoch < part2:
        lr = 0.001
        return lr
    if epoch >= part2:
        lr = 0.0005
        return lr

model.compile(
    optimizer=tf.keras.optimizers.Adam(lr=0.001),
    loss=tf.keras.losses.MeanSquaredError(),
#     loss=tf.keras.losses.MeanAbsoluteError(),
#     metrics=[tf.keras.metrics.MeanSquaredError()],
)

summary = model.summary()
print(summary)

callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

trace = model.fit(x=x_train, y=y_train, batch_size=128, epochs=runLen, verbose=1, 
                  validation_split=0.01, callbacks = [callback], shuffle=True)

#current best on combined dataset:
#   val_loss = 505
#   sum error on 10k test set: 0.97


In [ ]:
plt.subplot(2, 1, 1)
plt.plot(trace.history['loss'], '-')
plt.plot(trace.history['val_loss'], '-')
plt.legend(['train', 'val'], loc='upper left')
plt.xlabel('iteration')
plt.ylabel('loss')
# plt.ylim(10,1000)

In [7]:
#load larger test dataset (for calculating accuracy of network)

ft1 = "simulation/data/traj_9DOF_10k.txt"
ft2 = "simulation/data/jointPos_9DOF_10k.txt"
numTraj = 10000 #number of trajectories given in base file

tTest, jointPosTest = add_body_rotation(ft1, ft2, numTraj, mult =1)

saved augmented data as tbr, jbr


In [33]:
#load single test case (for use with viz)
#DEBUG -> why does this perform better when data is generated with fast restart disabled??

ft1 = "simulation/data/traj_9DOF_1.txt"
ft2 = "simulation/data/jointPos_9DOF_1.txt"
ft3 = "simulation/data/jointPath.txt"
numTraj = 1 #number of trajectories given in base file

tTest, jointPosTest = add_body_rotation(ft1, ft2, numTraj, mult =1, actual_traj=ft3)

# print(tTest)
# print(jointPosTest) #issue when only one element in jointPos input file

saved augmented data as tbr, jbr, at


In [34]:
#test model- IMPORTANT TO USE NEVER BEFORE SEEN DATA

x_test = tf.convert_to_tensor(tTest,np.float32)
# print(x_test)
y_test = tf.convert_to_tensor(jointPosTest,np.float32)

prediction = model.predict(x_test)
error = abs(y_test - prediction)

#average error for estimates for each joint
avg = np.average(error,axis=0)
print("average error = ", avg)

#range for each joint:
ranges = [50, 60, 67.5, 110, 120, 360, 130, 180, 110, 360]
rel_error = avg/ranges
print("error as frac of joint range = ", np.floor(rel_error*1000)/1000) #1 is full range of joint
print("total error = ",sum(rel_error))

print("prediction: ",prediction[0])
print("actual: ", y_test[0])

np.save("simulation/data/prediction.npy", prediction)

average error =  [ 7.4826236  6.666564   2.1891513  2.3657837  1.964421  12.34037
  0.8812828 21.157175   4.6880713 12.451317 ]
error as frac of joint range =  [0.149 0.111 0.032 0.021 0.016 0.034 0.006 0.117 0.042 0.034]
total error =  0.5668746287935109
prediction:  [ 7.5727600e-01  7.6436388e-01 -2.7448654e-02 -7.8315781e+01
  4.0157890e-01  8.0439369e+01 -5.3955284e+01  1.8372175e+01
  2.0578072e+01 -1.3837454e+02]
actual:  tf.Tensor(
[   8.2399     -5.9022     -2.2166    -75.95        2.366      68.099
  -53.074      -2.785      15.89     -125.923225], shape=(10,), dtype=float32)


Best total error: 

1.52 @ 10k  dataset, mult 1, BS 128, MSE

1.29 @ 100k dataset, mult 2, BS 128, MSE

1.15 @ 100k dataset, mult 1, BS 128, MSE

1.10 @ 100k dataset, mult 1, BS 64,  MSE, Val loss = 851, 10 epoch

0.97 @ 100k dataset, mult 1, BS 128,  MSE, Val loss = 623, 30 epoch

0.86 @ 100k dataset, mult 1, BS 128,  MSE, Val loss = 556, 100 epoch

0.90 @ 250k dataset (replace errors with repeats), mult 1, BS 128, val loss = 476, 100 epoch

0.82 @ combined 650k dataset, mult 1, BS 128, MSE, val loss = 371, 100 epoch

Notes:
artifially creating more data by rotating about y axis at different angles is a bad idea. This teaches the network to memorize patterns of movement rather than learn the underlying dynamic models.

In [ ]:
# model.save("10DOF.kmod")

In [2]:
#load best scoring model

model = tf.keras.models.load_model("10DOF.kmod")

In [ ]:
#proof my model is doing better than completely random guessing

np.random.seed(None)

# print(actual)
# print(tf.shape(actual)) #[99 7]
B = tf.random.uniform([1000,7])

# B = tf.ones([99,7])
B = B *tf.constant([25., 30., 33.75, 55. , 60., 180., 65.]) + tf.constant([0., 0., 26.25, -35., 30., 0., -65.])

# print(tf.shape(B))
# print(tf.shape(actual))

fake_error = (y_test - B)
# print(fake_error)

fake_avg = tf.math.reduce_mean(tf.math.abs(fake_error), axis=0)
print(fake_avg)

rel_fake_error = fake_avg/ranges

print("error as frac of joint range: ",rel_fake_error)
print("total error: ", sum(rel_fake_error))

#NOTE: these are not all the same becuase the starting ranges for joint positions do NOT fall in the middle of all
#      possible positions for each joint

In [ ]:
from scipy.spatial.transform import Rotation as R
r = R.from_euler('zyx', [
[90, 0, 0],
[0, 45, 0],
[45, 60, 30]], degrees=True)

print(r)
r.as_quat()

v = [1, 2, 3]

r.apply(v)

In [ ]:
r1 = R.from_euler('z', 90, degrees=True)
r2 = R.from_euler('x', 90, degrees=True)
r3 = r1*r2

v = [1, 2, 3]

rot1 = r1.apply(v)
rot2 = r3.apply(v)
print(rot2)


In [ ]:
a, b, c= r3.apply([1, 2, 3])
print(a,b,c)


In [ ]:
#Import data from MatLab SimScape Multibody Simulator

#data comes from two files
#1) n trajectories in xyz space, each length m
# traj = np.loadtxt(open("C:/Users/Derm/Desktop/traj_with_angs_1M.txt", "rb"), delimiter=",")
traj = np.loadtxt(open("simulation/data/traj_9DOF_100k.txt", "rb"), delimiter=",")

trajPts = np.shape(traj)[0] #points per trajectory
# numTraj = np.shape(traj)[1]//3 #number of total trajectories
numTraj = np.shape(traj)[1]//6 #number of total trajectories


#traj needs to be reshaped to a 3d numpy array
#as is traj[n] shows [x,y,z,x,y,z...]


#2) 7 joint angles at the end of the sequence
# jointPos = np.loadtxt(open("C:/Users/Derm/Desktop/jointPos_with_angs_1M.txt", "rb"), delimiter=",")
jointPos = np.loadtxt(open("simulation/data/jointPos_9DOF_100k.txt", "rb"), delimiter=",")

# print(traj[-1])
# print(jointPos[0])



In [ ]:
#reshape traj data into 3d numpy array
# t = np.zeros([trajPts,3,numTraj]) #net 1
# for j in range(np.shape(traj)[0]):
#     for i in range(np.shape(traj)[1]//3):
#         t[j,:,i] = traj[j,3*i:3*(i+1)]
        
t = np.zeros([trajPts,6,numTraj]) #net 3
for j in range(np.shape(traj)[0]):
    for i in range(np.shape(traj)[1]//6):
        t[j,:,i] = traj[j,6*i:6*(i+1)]

        
# print(t[:,:,0]) #same as in MatLab
print(np.shape(t))
#swap axis so batch size is first axis (for TF)
t = np.swapaxes(t,0,2)
print(np.shape(t)) #[numTraj, xyz, trajPts]
#swap axis again so that conv1D moves on time and not xyz
t = np.swapaxes(t,1,2)
print(np.shape(t))